<a href="https://colab.research.google.com/github/Ian-G27/Clase_MetodosNumericos/blob/main/Interpolacion_Hermite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpolacion de Hermite


In [2]:
import numpy as np
import pandas as pd
import sympy as sp
#Estos son los datos de entrada
x = np.array([1.3, 1.6, 1.9])               #nodos
f = np.array([0.6200860, 0.4554022, 0.2818186])  # f(xi)
df = np.array([-0.5220232, -0.5698959, -0.5811571])  # f'(xi)
n = len(x)
z = np.zeros(2 * n)
Q = np.zeros((2 * n, 2 * n))
# Paso 2:Duplicare los nodos y llenar f(x) y f'(x)
for i in range(n):
    z[2*i] = z[2*i+1] = x[i]
    Q[2*i, 0] = Q[2*i+1, 0] = f[i]
    Q[2*i+1, 1] = df[i]
    if i != 0:
        Q[2*i, 1] = (Q[2*i, 0] - Q[2*i-1, 0]) / (z[2*i] - z[2*i-1])
#Calculare las diferencias divididas
for i in range(2, 2*n):
    for j in range(2, i+1):
        Q[i, j] = (Q[i, j-1] - Q[i-1, j-1]) / (z[i] - z[i-j])
#Creare la tabla en pandas
nombre_columnas = ["i", "zᵢ", "Q[i,0]"] + [f"Q[i,{j}]" for j in range(1, 2*n)]
table_data = []
for i in range(2*n):
    row = [i, z[i], Q[i, 0]]
    for j in range(1, i+1):
        row.append(Q[i, j])
    table_data.append(row)
max_len = max(len(row) for row in table_data)
nombre_columnas = nombre_columnas[:max_len]
df_table = pd.DataFrame(table_data, columns=nombre_columnas)
print("\nTabla de interpolacion de Hermite:\n")
print(df_table.round(7))
#Por ultimo construire el polinomio
x_sym = sp.Symbol('x')
H = 0
term = 1
for i in range(2*n):
    H += Q[i, i] * term
    term *= (x_sym - z[i])
H_expanded = sp.expand(H)
print("\nPolinomio de Hermite H(x):")
sp.pprint(H_expanded, use_unicode=True)


Tabla de interpolacion de Hermite:

   i   zᵢ    Q[i,0]    Q[i,1]    Q[i,2]    Q[i,3]    Q[i,4]    Q[i,5]
0  0  1.3  0.620086       NaN       NaN       NaN       NaN       NaN
1  1  1.3  0.620086 -0.522023       NaN       NaN       NaN       NaN
2  2  1.6  0.455402 -0.548946 -0.089743       NaN       NaN       NaN
3  3  1.6  0.455402 -0.569896 -0.069833  0.066366       NaN       NaN
4  4  1.9  0.281819 -0.578612 -0.029054  0.067966  0.002667       NaN
5  5  1.9  0.281819 -0.581157 -0.008484  0.068567  0.001002 -0.002775

Polinomio de Hermite H(x):
                       5                       4                       3       ↪
- 0.00277469135798944⋅x  + 0.0240317901231809⋅x  - 0.0145560802460553⋅x  - 0.2 ↪

↪                 2                                           
↪ 35216169754413⋅x  - 0.00822922345577151⋅x + 1.00194406469105


# Explicacion del codigo

Librerias utilizadas



```
import numpy as np   #Para hacer operaciones matematicas
import pandas as pd  #Para imprimir una tabla como la del libro
import sympy as sp   #Para manipular y mostrar simbólicamente el polinomio de Hermite

```



Datos de entrada

$x:$ son los puntos conocidos

$f:$ son los valores de ola funcion en esos puntos

$df:$ son las derivadas de la funcion en los puntos

$n:$ es el numero de puntos

Paso 1,2 y 3 del pseudocodigo




```
# Crear z y Q
z = np.zeros(2 * n)
Q = np.zeros((2 * n, 2 * n))
for i in range(n):
    z[2 * i] = x[i]
    z[2 * i + 1] = x[i]
    Q[2 * i][0] = f[i]
    Q[2 * i + 1][0] = f[i]
    Q[2 * i + 1][1] = df[i]
    if i != 0:
        Q[2 * i][1] = (Q[2 * i][0] - Q[2 * i - 1][0]) / (z[2 * i] - z[2 * i - 1])
```

$z$ duplica cada valor de $x$ ya que el algoritmo repite los nodos

$Q[:,0]$ se llena con los valores de $f(x)$

$Q[impar,1]$ se llena con $f´[x]$

Para los nodos repetidos se usa la formula del paso del pseudocodigo(diferencia dividida cuando hay repeticion de los nodos)



Paso 4: calculando las diferencias divididas


```
for i in range(2, 2*n):
    for j in range(2, i+1):
        Q[i, j] = (Q[i, j-1] - Q[i-1, j-1]) / (z[i] - z[i-j])
```
Se llenan el resto de las diferencias divididas usando la formula general de Newton-Hermite para construir la matriz triangular.


Paso 5: Construccion de la tabla para que sea visualizada


```
column_names = ["i", "zᵢ", "Q[i,0]"] + [f"Q[i,{j}]" for j in range(1, 2*n)]
table_data = []
for i in range(2*n):
    row = [i, z[i], Q[i, 0]]
    for j in range(1, i+1):
        row.append(Q[i, j])
    table_data.append(row)
```
Se genera la tabla paso a paso, agregando las entradas de cada fila de Q hasta

*   Se genera la tabla paso a paso, agregando las entradas de cada fila de Q hasta la columna correspondiente.


```
max_len = max(len(row) for row in table_data)
column_names = column_names[:max_len]
df_table = pd.DataFrame(table_data, columns=column_names)

print("\nTabla de interpolación de Hermite:\n")
print(df_table.round(7))
```
Se ajusta el número de columnas para el DataFrame.

*   Se ajusta el número de columnas para el DataFrame.
*   Se imprime la tabla redondeada a 7 decimales.

Paso 6:Construcción simbólica del polinomio de Hermite



```
x_sym = sp.Symbol('x')
H = 0
term = 1
for i in range(2*n):
    H += Q[i, i] * term
    term *= (x_sym - z[i])

```
*  Se construye simbólicamente el polinomio de Hermite $H(x)$ con la fórmula: $H(x)= Q_{0,0}+Q_{1,1}(x-z_0)(x-z_1)+....$



```
H_expanded = sp.expand(H)
print("\nPolinomio de Hermite H(x):")
sp.pprint(H_expanded, use_unicode=True)

```

*  Se expande el polinomio en forma estándar (multiplicando los factores).
*  Se imprime el polinomio en forma legible usando SymPy.